In [13]:
import pandas as pd


def normalize_trade_date(value):
    """Normalize any incoming date to a single string format '%m/%d/%Y'."""
    if value is None:
        return None

    # Try pandas first (handles many formats)
    try:
        dt = pd.to_datetime(value, errors="coerce")
        if not pd.isna(dt):
            return dt.strftime("%m/%d/%Y")
    except Exception:
        pass

    from datetime import datetime
    s = str(value).strip()
    if not s:
        return None

    for fmt in ("%Y-%m-%d", "%d/%m/%Y", "%m/%d/%Y", "%m-%d-%Y"):
        try:
            dt = datetime.strptime(s, fmt)
            return dt.strftime("%m/%d/%Y")
        except ValueError:
            continue

    return None


def generate_trades(csv_file, portfolio_type='simple'):
    """
    Generate trade code from CSV.

    Args:
        csv_file: Path to CSV file. It must have:
            - one of: ticker, symbol, tvId, tv_Id
            - side, price, quantity columns
            - optional date/cts/mts column for trade date
        portfolio_type: 'simple' or 'full'
    """
    df = pd.read_csv(csv_file)

    # Build case-insensitive column lookup
    cols = {str(c).lower(): c for c in df.columns}

    def get_col(candidates, required=True, label=None):
        """Return the first matching column name from candidates (case-insensitive)."""
        label = label or candidates
        for cand in candidates:
            key = str(cand).lower()
            if key in cols:
                return cols[key]
        if required:
            raise ValueError(
                f"CSV must contain column(s) {candidates} for {label} (case-insensitive). "
                f"Found columns: {list(df.columns)}"
            )
        return None

    # Core columns
    ticker_col   = get_col(["ticker", "symbol", "tvid", "tv_id"], label="ticker")
    side_col     = get_col(["side"], label="side")
    price_col    = get_col(["price"], label="price")
    quantity_col = get_col(["quantity"], label="quantity")
    date_col     = get_col(["date", "cts", "mts"], required=False, label="date")

    print("# Reset portfolio")
    print("reset_portfolio()\n")

    for _, row in df.iterrows():
        ticker = str(row[ticker_col]).lower()
        side   = str(row[side_col]).lower()
        price  = float(row[price_col])
        qty    = abs(float(row[quantity_col]))

        # Optional date column; normalize if present
        trade_date = normalize_trade_date(row[date_col]) if date_col is not None else None

        if portfolio_type == 'simple':
            if trade_date is not None:
                print(
                    f"add_trade(ticker='{ticker}', side='{side}', "
                    f"price={price}, quantity_buy={qty}, date='{trade_date}')"
                )
            else:
                print(
                    f"add_trade(ticker='{ticker}', side='{side}', "
                    f"price={price}, quantity_buy={qty})"
                )
        else:  # full
            if trade_date is not None:
                print(
                    f"add_trade(ticker='{ticker}', asset_type='Equity', side='{side}', "
                    f"price={price}, quantity_buy={qty}, date='{trade_date}')"
                )
            else:
                print(
                    f"add_trade(ticker='{ticker}', asset_type='Equity', side='{side}', "
                    f"price={price}, quantity_buy={qty})"
                )

    print("\n# Display")
    print("df = get_portfolio_df()")
    print("df")

In [14]:
# Example usage for portfolio_simple.ipynb
generate_trades('questdb-query-1764846052060.csv')

# Reset portfolio
reset_portfolio()

add_trade(ticker='eth_crypto', side='buy', price=2797.381878, quantity_buy=1.0, date='12/02/2025')
add_trade(ticker='eth_crypto', side='hold', price=2997.783701, quantity_buy=1.0, date='12/02/2025')
add_trade(ticker='xrp_crypto', side='buy', price=2.005788241, quantity_buy=3000.0, date='12/02/2025')
add_trade(ticker='xrp_crypto', side='hold', price=2.005788241, quantity_buy=3000.0, date='12/02/2025')
add_trade(ticker='aapl_nasdaq', side='buy', price=282.81, quantity_buy=2.0, date='12/02/2025')
add_trade(ticker='aapl_nasdaq', side='hold', price=282.81, quantity_buy=2.0, date='12/02/2025')
add_trade(ticker='xrp_crypto', side='buy', price=2.043734885, quantity_buy=1.0, date='12/02/2025')
add_trade(ticker='xrp_crypto', side='hold', price=2.043734885, quantity_buy=1.0, date='12/02/2025')
add_trade(ticker='aapl_nasdaq', side='buy', price=282.68, quantity_buy=20.0, date='12/02/2025')
add_trade(ticker='aapl_nasdaq', side='hold', price=282.68, quantity_buy=2